### Q1 Getting the embeddings model

In [4]:
# Import libraries
from sentence_transformers import SentenceTransformer

# Get the model
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

# Define the question
user_question = "I just discovered the course. Can I still join it?"

# Create a vector
q_vector = embedding_model.encode(user_question)

print("Answer 1: ", q_vector[0])

Answer 1:  0.078222655


In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()